# Carbon Investment Potential
Data preparation for the carbon investment potential widget (pie/donut chart).  
Data model is as follows:  

location_id: staging API id  
category: carbon_5, carbon_10, protected, reaining 
area: int  
year: 2016 (see in code)   
description: area values as string (with error in 5$ and 10$)  

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import requests
import os


In [2]:
data = pd.read_csv('https://storage.googleapis.com/mangrove_atlas/widget_data/Investible_Blue_Carbon_Update_20220725.csv')
#data = pd.read_csv('../../../../data/Investible_Blue_Carbon_Update_20220725.csv')
data.head()


,id,type,iso,name,Investible Blue Carbon Extent @ $5/ton,Investible Blue Carbon Extent @ $5/ton.1,Investible Blue Carbon Extent @ $10/ton,Investible Blue Carbon Extent @ $10/ton.1,Investible Blue Carbon Extent (Diff btw. $10/ton and $5/ton),Extent within PA,Remaining extent,Extent,Include?
0,NaN,country,ABW,Aruba,0,0.0,0,0.0,0.0,1.0,45.0,46.0,N
1,1_2_97,country,AGO,Angola,900 (_+100),900.0,"1,300 (_+ 100)",1300.0,400.0,146.0,26911.0,28357.0,Y
2,NaN,country,AIA,Anguilla,0,0.0,0,0.0,0.0,0.0,4.0,4.0,N
3,1_2_61,country,ARE,United Arab Emirates,0,0.0,0,0.0,0.0,1323.0,6122.0,7445.0,N
4,NaN,country,ASM,American Samoa,0,0.0,0,0.0,0.0,0.0,32.0,32.0,N


In [6]:
data_clean = data[data['Include?'] == 'Y'][["iso","Investible Blue Carbon Extent @ $5/ton", "Investible Blue Carbon Extent @ $10/ton" ,"Investible Blue Carbon Extent @ $5/ton.1","Investible Blue Carbon Extent @ $10/ton.1","Extent within PA","Remaining extent"]].copy()
data_clean.columns = ['iso', 'carbon_5_text','carbon_10_text', 'carbon_5', 'carbon_10', 'protected', 'remaining']
data_clean['carbon_5_text'] = data_clean['carbon_5_text'].str.replace('_\+', '±').str.replace('\t', '')
data_clean['carbon_10_text'] = data_clean['carbon_10_text'].str.replace('_\+', '±').str.replace('\t', '')
data_clean.head()



/var/folders/wf/_wlxc6cn5js4hh3j7j6x28f80000gn/T/ipykernel_2579/1557417207.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  data_clean['carbon_5_text'] = data_clean['carbon_5_text'].str.replace('_\+', '±').str.replace('\t', '')
/var/folders/wf/_wlxc6cn5js4hh3j7j6x28f80000gn/T/ipykernel_2579/1557417207.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  data_clean['carbon_10_text'] = data_clean['carbon_10_text'].str.replace('_\+', '±').str.replace('\t', '')


,iso,carbon_5_text,carbon_10_text,carbon_5,carbon_10,protected,remaining
1,AGO,900 (±100),"1,300 (± 100)",900.0,1300.0,146.0,26911.0
10,BGD,"98,900 (±2,800)","113,700 (± 3,200)",98900.0,113700.0,397048.0,0.0
13,BLZ,300 (±100),"1,100 (± 400)",300.0,1100.0,15868.0,35901.0
15,BRA,"17,300 (±4,900)","49,900 (± 14,100)",17300.0,49900.0,960634.0,130937.0
20,CMR,"33,500 (±1,100)","38,300 (± 1,300)",33500.0,38300.0,149115.0,9586.0


In [7]:
data_clean.columns

Index(['iso', 'carbon_5_text', 'carbon_10_text', 'carbon_5', 'carbon_10',
       'protected', 'remaining'],
      dtype='object')

In [9]:
carbon_data = data_clean[['iso', 'carbon_5', 'carbon_10', 'protected', 'remaining']].copy()
carbon_data_long = pd.melt(carbon_data, id_vars='iso')
carbon_data_long

,iso,variable,value
0,AGO,carbon_5,900.0
1,BGD,carbon_5,98900.0
2,BLZ,carbon_5,300.0
3,BRA,carbon_5,17300.0
4,CMR,carbon_5,33500.0
...,...,...,...
199,SUR,remaining,15964.0
200,THA,remaining,214805.0
201,TZA,remaining,4689.0
202,VEN,remaining,87933.0


In [13]:
carbon_text = data_clean[['iso', 'carbon_5_text', 'carbon_10_text', 'protected', 'remaining']].copy()
carbon_text.rename(columns={'carbon_5_text':'carbon_5', 'carbon_10_text':'carbon_10'}, inplace=True)
carbon_text['protected'] = carbon_text['protected'].map('{:,.0f}'.format)
carbon_text['remaining'] = carbon_text['remaining'].map('{:,.0f}'.format)
carbon_text.head()

,iso,carbon_5,carbon_10,protected,remaining
1,AGO,900 (±100),"1,300 (± 100)",146,"26,911"
10,BGD,"98,900 (±2,800)","113,700 (± 3,200)","397,048",0
13,BLZ,300 (±100),"1,100 (± 400)","15,868","35,901"
15,BRA,"17,300 (±4,900)","49,900 (± 14,100)","960,634","130,937"
20,CMR,"33,500 (±1,100)","38,300 (± 1,300)","149,115","9,586"


In [16]:
carbon_text_long = pd.melt(carbon_text, id_vars='iso')
carbon_text_long.rename(columns={'value':'note'}, inplace=True)
carbon_text_long.head(10)

,iso,variable,note
0,AGO,carbon_5,900 (±100)
1,BGD,carbon_5,"98,900 (±2,800)"
2,BLZ,carbon_5,300 (±100)
3,BRA,carbon_5,"17,300 (±4,900)"
4,CMR,carbon_5,"33,500 (±1,100)"
5,COL,carbon_5,"5,700 (±1,500)"
6,CRI,carbon_5,200 (±100)
7,CUB,carbon_5,"10,000 (±5,400)"
8,CYM,carbon_5,200 (±100)
9,DOM,carbon_5,200 (±100)


In [17]:
#Old version of note (full 5$ sentence)
#carbon_join = pd.merge(carbon_data_long, data_clean[['iso', 'sentence_value']], on = 'iso', how = 'left')
#carbon_join['note'] = [f'extent of investible blue carbon (ha) is {x} (at $5/ton)' for x in carbon_join["sentence_value"].values]

#New version (text values of areas)
carbon_join = pd.merge(carbon_data_long, carbon_text_long, on = ['iso', 'variable'], how='inner')
carbon_join

,iso,variable,value,note
0,AGO,carbon_5,900.0,900 (±100)
1,BGD,carbon_5,98900.0,"98,900 (±2,800)"
2,BLZ,carbon_5,300.0,300 (±100)
3,BRA,carbon_5,17300.0,"17,300 (±4,900)"
4,CMR,carbon_5,33500.0,"33,500 (±1,100)"
...,...,...,...,...
199,SUR,remaining,15964.0,"15,964"
200,THA,remaining,214805.0,"214,805"
201,TZA,remaining,4689.0,"4,689"
202,VEN,remaining,87933.0,"87,933"


Add locations

In [18]:
locations_file = 'https://storage.googleapis.com/mangrove_atlas/boundaries/processed/location_final/locations_v3_not_merged_with_old.gpkg'
locations = gpd.read_file(locations_file)
locations = locations[locations['type'] == 'country']
locations.head()

,name,iso,type,area_m2,wdpaid,globalid,perimeter_m,location_idn,coast_length_m,geometry
82,Qatar,QAT,country,3.880224,NaN,{AF97ABE2-6405-4438-A7ED-1494A43DA379},8.392644,06d2e6f9-bc89-59bf-a0e2-ab804e5db9fd,1345769.96,"MULTIPOLYGON (((50.73769 24.93464, 50.73779 24..."
89,Mayotte,MYT,country,5.611808,NaN,{57E86B5B-7EF0-4754-A8D4-A9DC3212D421},10.086238,0750953f-4af9-549b-aeea-329663249a56,291036.71,"POLYGON ((46.63483 -12.96039, 46.63197 -12.969..."
118,Vietnam,VNM,country,90.156489,NaN,{B2A84FBB-34CD-4A51-9463-B9DB2DB62A10},81.714911,09a1ab14-11ad-56ec-8acb-a149e5697abd,9005760.08,"MULTIPOLYGON (((104.31952 10.36051, 104.31975 ..."
132,Grenada,GRD,country,2.154728,NaN,{F8753179-5FFA-4D9E-8AD9-083F31C48528},6.743601,0b0ecb56-bb8e-5ef1-b8ee-3cdad67fed0e,260664.47,"MULTIPOLYGON (((-61.91525 11.37330, -61.91813 ..."
149,India,IND,country,473.029671,NaN,{A4A6CE4D-8D03-4246-9A2F-BD9811232115},211.564078,0c07ca53-7b17-5650-a2c6-0cc27249a4bd,16917891.22,"MULTIPOLYGON (((79.52922 9.38411, 79.52921 9.3..."


In [19]:
api_locs = pd.read_csv('https://storage.googleapis.com/mangrove_atlas/widget_data/locations_staging.csv')
api_locs.rename(columns={'location_id': 'location_idn'}, inplace=True)
api_locs.head()

,id,location_idn
0,1563,000bd204-c0fd-510b-a1ad-132a7ef7470d
1,1564,00250a0f-f66d-54a0-b7a3-d80035881cbf
2,1565,0041637b-f6a2-5b89-87ce-850f5c5431b3
3,1566,005b49ef-6b7f-575a-85b3-ff19261a0755
4,1567,00921349-70fb-5a7e-8207-b3157aecc349


In [20]:
df_final = pd.merge(carbon_join, locations[['iso', 'location_idn']], on='iso', how='left')
df_final = pd.merge(df_final, api_locs, on='location_idn', how='left')
df_final.head()

,iso,variable,value,note,location_idn,id
0,AGO,carbon_5,900.0,900 (±100),27ceab8c-946e-5286-a06f-8bd98ec81f77,2029
1,BGD,carbon_5,98900.0,"98,900 (±2,800)",f58d8175-a251-50e8-b1e1-756d60d70d96,4587
2,BLZ,carbon_5,300.0,300 (±100),b686e902-4ce4-5b7a-b84a-e76021dd3f03,3831
3,BRA,carbon_5,17300.0,"17,300 (±4,900)",2381ce0a-de27-5ee6-85fe-08a57acb21f0,1985
4,CMR,carbon_5,33500.0,"33,500 (±1,100)",ee9cec4a-46d2-5d26-a424-7b6be55f1ed3,4493


In [21]:
df_final.drop(columns=['iso', 'location_idn'], inplace=True)
df_final.rename(columns={'id':'location_id', 'variable':'category', 'note':'description', 'value':'area'}, inplace=True)
#df_final['unit'] = 'ha'
df_final.head()

,category,area,description,location_id
0,carbon_5,900.0,900 (±100),2029
1,carbon_5,98900.0,"98,900 (±2,800)",4587
2,carbon_5,300.0,300 (±100),3831
3,carbon_5,17300.0,"17,300 (±4,900)",1985
4,carbon_5,33500.0,"33,500 (±1,100)",4493


In [22]:
df_final[df_final.area.isnull()]

,category,area,description,location_id
68,carbon_10,NaN,NaN,1695


In [23]:
len(df_final)

204

In [24]:
df_final = df_final[df_final.area.notnull()]
len(df_final)

203

In [25]:
df_final.to_csv('../../../../data/investment_potential_widget_UPDATE.csv', index=False)